In [1]:
!pip install password_strength
!pip install chars2vec

     |████████████████████████████████| 8.1MB 3.4MB/s 
  Created wheel for chars2vec: filename=chars2vec-0.1.7-cp36-none-any.whl size=8111095 sha256=8b8de67cdd11d5b6f12e395c341bcb85bc7536abaec7729645d38cb7ad1d6f25
  Stored in directory: /root/.cache/pip/wheels/97/b6/65/d7e778ef1213ec77d315aea0f536068b96e36cc94c02abbfde
Successfully built chars2vec


In [2]:
import numpy as np
import pandas as pd
import chars2vec
from pathlib import Path
from password_strength import PasswordStats
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.feature_extraction import FeatureHasher

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls drive/My\ Drive/dmia

ctb_pred.csv	       test_w_groups.csv   train_w_groups.csv
passwords.ipynb        test_wt_groups.csv  train_wt_groups.csv
sample_submission.csv  train.csv	   Xtest.csv


In [0]:
PATH_TO_DATA = Path('drive/My Drive/dmia')
# !cd .. && ls input/dmia-sport-2019-fall-intro

In [0]:
train = pd.read_csv(PATH_TO_DATA / 'train.csv')
test = pd.read_csv(PATH_TO_DATA / 'Xtest.csv')

In [0]:
train.shape

(4151496, 2)

In [0]:
train['Password'] = train['Password'].fillna('')
test['Password'] = test['Password'].fillna('')

In [0]:
def statsPassword(data):
    data['alphabet_cardinality'] = data['Password'].apply(lambda x: PasswordStats(x).alphabet_cardinality)
    data['entropy_bits'] = data['Password'].apply(lambda x: PasswordStats(x).entropy_bits if len(x) > 0 else 0)
    data['length'] = data['Password'].apply(lambda x: PasswordStats(x).length)
    data['Ll'] = data['Password'].apply(lambda x: PasswordStats(x).letters_lowercase)
    data['repeated_patterns_length'] = data['Password'].apply(lambda x: PasswordStats(x).repeated_patterns_length)
    data['Nd'] = data['Password'].apply(lambda x: PasswordStats(x).numbers)
    data['sequences_length'] = data['Password'].apply(lambda x: PasswordStats(x).sequences_length)
    data['strength'] = data['Password'].apply(lambda x: PasswordStats(x).strength() if len(x) > 0 else 0)
    data['weakness_factor'] = data['Password'].apply(lambda x: PasswordStats(x).weakness_factor if len(x) > 0 else 1)
    
    return data
    




In [0]:
%%time
X = statsPassword(train)
test = statsPassword(test).drop(['Id'], axis=1)
X.shape, test.shape

CPU times: user 8min 35s, sys: 437 ms, total: 8min 35s
Wall time: 8min 36s


In [0]:
X.shape, test.shape

((4151496, 11), (1037875, 10))

In [0]:
X.head()

,Password,Times,alphabet_cardinality,entropy_bits,length,Ll,repeated_patterns_length,Nd,sequences_length,strength,weakness_factor
0,631XniVx2lS5I,2,13,48.105716,13,4,0,5,0,0.604087,0.0
1,LEGIT747,1,7,22.458839,8,0,0,3,0,0.249543,0.0
2,742364es,1,7,22.458839,8,2,0,6,0,0.249543,0.0
3,3846696477,1,6,25.849625,10,0,4,10,0,0.287218,0.4
4,laurahop,2,7,22.458839,8,8,0,0,0,0.249543,0.0


In [0]:
X.to_csv(PATH_TO_DATA / 'train_wt_groups.csv', index=False)
test.to_csv(PATH_TO_DATA / 'test_wt_groups.csv', index=False)

In [0]:
train = pd.read_csv(PATH_TO_DATA / 'train_wt_groups.csv')
test = pd.read_csv(PATH_TO_DATA / 'test_wt_groups.csv')

In [0]:
train.loc[train['Password'] == '', 'Times'] = 30
train = train.drop(157331)
train = train.reset_index(drop=True)

In [0]:
for col in train:
    if sum(train[col].isna()) > 0:
        print(col)

In [0]:
from sklearn.model_selection import KFold
import numpy as np

class TrainModel:
    def __init__(self, X, y, test, model, metric, isCat=True, test_size=0.3, random_state=42):
        self.X = X.data.reset_index(drop=True).values
        self.y = np.log(y.reset_index(drop=True).values + 1)
        self.cols = X.data.columns
        self.test = test.data
        self.model = model
        self.isCat = isCat
        self.test_size = test_size
        self.metric = metric
        self.random_state = random_state
        self.oof = np.zeros(len(test.data))
        self.kfolds = np.zeros(len(X.data))
        self.error = 0
        self.features_importances = np.zeros(X.data.shape[1])

    def fitModel(self, n_splits=5):
        kfolds = KFold(n_splits=n_splits)
        for train_idx, test_idx in kfolds.split(self.X):
            X_train, X_test = self.X[train_idx], self.X[test_idx]
            y_train, y_test = self.y[train_idx], self.y[test_idx]
            self.model.fit(X_train, y_train)
            preds_val = np.exp(self.model.predict(X_test)) - 1
            
            self.kfolds[test_idx] += preds_val
            error = self.metric(list(np.exp(y_test) - 1), list(preds_val))
            print('eval_metric: ', error)
            self.error += error
            preds_test = np.exp(self.model.predict(self.test)) - 1
            
            self.oof += preds_test
            if self.isCat: 
                self.features_importances += self.model.feature_importances_
        self.error /= n_splits
        self.oof /= n_splits
        self.features_importances /= n_splits
  
    def getFeaturesImportances(self):
        data = pd.DataFrame(self.features_importances, index=self.cols).sort_values(by=0, ascending=False)
        data.columns = ['features_importance']
        return data

    def getError(self):
        return self.error 

    def getOof(self):
        return self.oof

    def getKfoldsPreds(self):
        return self.kfolds

In [0]:
from scipy.spatial.distance import euclidean

def getSubmission(preds, fileName):
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='Id')
    sample_sub['Times'] = preds
    sample_sub.to_csv(PATH_TO_DATA / fileName)

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(np.log(y_pred[i] + 1) - np.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

def keyBoardDistances(keyA, keyB):
    coordsA = _getCoords(keyA)
    coordsB = _getCoords(keyB)

    return euclidean(coordsA, coordsB)
  
def _getCoords(key):
    keyBoardLines = ['`1234567890-=', 
                'qwertyuiop[]',
                'asdfghjkl;\'\\',
                '<zxcvbnm,./']
    keyBoardLinesSplitted = [list(line) for line in keyBoardLines]
    for lineIdx, line in enumerate(keyBoardLinesSplitted):
        try:
            charIdx = line.index(key)
        return [lineIdx, charIdx]
            except ValueError:
        continue
    return [0, 0]

In [0]:
X = train.drop('Times', axis=1)
y = train['Times']
X.shape, test.shape

((4151495, 10), (1037875, 10))

In [0]:
X.head()

,Password,alphabet_cardinality,entropy_bits,length,Ll,repeated_patterns_length,Nd,sequences_length,strength,weakness_factor
0,631XniVx2lS5I,13,48.105716,13,4,0,5,0,0.604087,0.0
1,LEGIT747,7,22.458839,8,0,0,3,0,0.249543,0.0
2,742364es,7,22.458839,8,2,0,6,0,0.249543,0.0
3,3846696477,6,25.849625,10,0,4,10,0,0.287218,0.4
4,laurahop,7,22.458839,8,8,0,0,0,0.249543,0.0


In [0]:
X.loc[:, 'percent_nums'] = X['Nd'] / X['length']
test.loc[:, 'percent_nums'] = test['Nd'] / test['length']

In [0]:
X.loc[:, 'percent_letters'] = X['Ll'] / X['length']
test.loc[:, 'percent_letters'] = test['Ll'] / test['length']

In [0]:
X.loc[:, 'strength_fin'] = (1 - X['weakness_factor']) * X['strength']
test.loc[:, 'strength_fin'] = (1 - test['weakness_factor']) * test['strength']

In [0]:
X.loc[:, 'diff_alpabet_cardinality'] = (X['length'] - X['alphabet_cardinality']) / X['length']
test.loc[:, 'diff_alpabet_cardinality'] = (test['length'] - test['alphabet_cardinality']) / test['length']

In [0]:
X.loc[:, 'diff_Ll'] = (X['length'] - X['Ll']) / X['length']
test.loc[:, 'diff_Ll'] = (test['length'] - test['Ll']) / test['length']

In [0]:
X.loc[:, 'diff_Nd'] = (X['length'] - X['Nd']) / X['length']
test.loc[:, 'diff_Nd'] = (test['length'] - test['Nd']) / test['length']

In [0]:
X.loc[:, 'uniqness'] = X['Password'].apply(lambda x: len(''.join(set(x)))) / X['length']
test.loc[:, 'uniqness'] = test['Password'].fillna('').apply(lambda x: len(''.join(set(x)))) / test['length']


In [0]:
X.shape, test.shape

((4151495, 17), (1037875, 17))

In [0]:
X = X.fillna(0.0)
test = test.fillna(0.0)

In [0]:
X['cat_strength'] = pd.cut(X['strength'], 9, labels=list(range(9)))
test['cat_strength'] = pd.cut(test['strength'], 9, labels=list(range(9)))

In [0]:
X_w_target = X.copy()
X_w_target['Times'] = y
print(X_w_target.groupby(by='cat_strength').agg('mean')['Times'])
print(X_w_target['cat_strength'].value_counts(normalize=True))

cat_strength
high_medium    1.040267
high_weak      1.590057
low_medium     1.166987
low_strong     1.032608
low_weak       5.091072
medium         1.097602
strong         1.014164
very_strong    1.019517
weak           2.427595
Name: Times, dtype: float64
weak           0.400905
high_weak      0.384312
low_medium     0.089121
medium         0.048952
high_medium    0.033541
low_weak       0.024812
low_strong     0.013910
strong         0.003928
very_strong    0.000518
Name: cat_strength, dtype: float64


In [0]:
X[X['Password'].isna()]

,Password,alphabet_cardinality,entropy_bits,length,Ll,repeated_patterns_length,Nd,sequences_length,strength,weakness_factor


In [0]:
%%time
from sklearn.decomposition import PCA

feat_hasher = FeatureHasher(n_features=100, input_type='string')

pca = PCA(n_components=10, tol=0.3, svd_solver='randomized', random_state=42)
train_latent_features = pca.fit_transform(feat_hasher.fit_transform(X['Password']).toarray())
test_latent_features = pca.transform(feat_hasher.transform(test['Password']).toarray())

CPU times: user 1min 10s, sys: 12 s, total: 1min 22s
Wall time: 45.2 s


In [0]:
train_latent_features.shape

(4151495, 10)

In [0]:
train_latent_features = pd.DataFrame(train_latent_features, columns=[f'latent_{i}' for i in range(1, train_latent_features.shape[1] + 1)])
test_latent_features = pd.DataFrame(test_latent_features, columns=[f'latent_{i}' for i in range(1, test_latent_features.shape[1] + 1)])

In [0]:
X = pd.concat((X, train_latent_features), axis=1)
test = pd.concat((test, test_latent_features), axis=1)

In [0]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
# lgb = LGBMRegressor(**param)

In [0]:
### training by groups with stacking

# # !pip install catboost
# # from catboost import CatBoostRegressor
# from lightgbm import LGBMRegressor
# # catboost = CatBoostRegressor(learning_rate=0.01, n_estimators=2000, task_type='GPU', silent=True)
# lgb = LGBMRegressor(**param)
# ridge = Ridge(alpha=0.3)
# reg = LinearRegression()
# lasso = Lasso(0.4)
# # forest = RandomForestRegressor(n_estimators=100, max_depth=5)



# preds = np.zeros(len(test))
# for cat_strength in X['cat_strength'].unique():
#   print(f'Training {cat_strength}')
#   X_tmp = X[X['cat_strength'] == cat_strength].drop(['Password', 'cat_strength'], axis=1).copy()
#   # X_tmp = scaleData(X_tmp)
#   train_idx = X_tmp.index
#   y_tmp = y[train_idx].copy()
#   test_tmp = test[test['cat_strength'] == cat_strength].drop(['Password', 'cat_strength'], axis=1).copy()
#   # test_tmp = scaleData(test_tmp)
#   test_idx = test_tmp.index
#   for i, model in enumerate([ridge, reg, lasso]):
#     print(f'Train model #{i+1}')
#     trainer = TrainModel(pd.DataFrame(X_tmp), y_tmp, pd.DataFrame(test_tmp), model, rmsle, isCat=False)
#     trainer.fitModel()
#     # print('shapes',X_tmp.shape, y_tmp.shape, test_tmp.shape)
#     X_tmp = np.concatenate((X_tmp, trainer.getKfoldsPreds().reshape(-1, 1)), axis=1)
#     test_tmp = np.concatenate((test_tmp, trainer.getOof().reshape(-1, 1)), axis=1)
#   print('\nTrain model with stacking: ')
#   trainer = TrainModel(pd.DataFrame(X_tmp), y_tmp, pd.DataFrame(test_tmp), lgb, rmsle, isCat=True)
#   trainer.fitModel()
#   print('Mean metric: ', trainer.getError(), '\n')
#   preds[test_idx] += trainer.getOof()

In [0]:
X.head()

,Password,alphabet_cardinality,entropy_bits,length,Ll,repeated_patterns_length,Nd,sequences_length,strength,weakness_factor,percent_nums,percent_letters,strength_fin,diff_alpabet_cardinality,diff_Ll,diff_Nd,uniqness,cat_strength
0,631XniVx2lS5I,13,48.105716,13,4,0,5,0,0.604087,0.0,0.384615,0.307692,0.604087,0.000,0.692308,0.615385,1.000,5
1,LEGIT747,7,22.458839,8,0,0,3,0,0.249543,0.0,0.375000,0.000000,0.249543,0.125,1.000000,0.625000,0.875,2
2,742364es,7,22.458839,8,2,0,6,0,0.249543,0.0,0.750000,0.250000,0.249543,0.125,0.750000,0.250000,0.875,2
3,3846696477,6,25.849625,10,0,4,10,0,0.287218,0.4,1.000000,0.000000,0.172331,0.400,1.000000,0.000000,0.600,2
4,laurahop,7,22.458839,8,8,0,0,0,0.249543,0.0,0.000000,1.000000,0.249543,0.125,0.000000,1.000000,0.875,2


In [0]:
X.shape, test.shape, y.shape

((4151495, 20), (1037875, 20), (4151495,))

In [0]:
from lightgbm import LGBMRegressor, Dataset

X_dropped = X.drop('Password', axis=1)
test_dropped = test.drop('Password', axis=1)
X_dataset_lgb = Dataset(X_dropped, categorical_feature=[X_dropped.shape[1] - 11])
test_dataset_lgb = Dataset(test_dropped, categorical_feature=[X_dropped.shape[1] - 11])
lgb = LGBMRegressor(**param)
trainer = TrainModel(X_dataset_lgb, y, test_dataset_lgb, lgb, rmsle)
trainer.fitModel()

eval_metric:  0.38802604766966975
eval_metric:  0.3837803061211211
eval_metric:  0.3860239025109649
eval_metric:  0.38739136582196393
eval_metric:  0.38563421025710676


In [0]:
trainer.getError()

0.38617116647616523

In [0]:
trainer.getFeaturesImportances()

,features_importance
length,556.2
latent_3,377.8
entropy_bits,248.2
percent_letters,237.0
latent_8,229.4
Nd,143.0
latent_7,115.0
latent_10,113.6
latent_1,108.6
percent_nums,93.6


In [0]:
getSubmission(trainer.getOof(), 'submission.csv')